In [2]:
# Utility Functions

import csv
import os
from tkinter.filedialog import askopenfilename
from tkinter import Tk

def get_filepath(filename=None):
    return os.path.join(os.getcwd(), '..', 'data', filename)

def getfile(initialdir=None, title=None):
    Tk().withdraw()
    
    filename = askopenfilename(initialdir=initialdir or os.path.expanduser("~"),
                               title=title or 'Select report file')
    
    return filename

def print_summary(summary):
    for key, value in summary.items():
        print(key, ':', 
              value, '/', summary['total'], 
              '(', round(value / summary['total'] * 100, 2), '%)')


In [8]:
#! /usr/bin/env python3

# This script analyses the diabetes data produced by the search
# MOH - Diabetes - Performance Measures

targets = {
        'Months since Hb A1C': 6,
        'Months since Systolic BP [BP:]': 6,
        'Months since LDL': 12,
}

summary = {
        'Months since Hb A1C': 0,
        'Months since Systolic BP [BP:]': 0,
        'Months since LDL': 0,
        'All': 0,
        'total': 0
}

filename = getfile(title="Select diabetes report")

with open(filename) as file:
    next(file)

    reader = csv.DictReader(file)

    for row in reader:
        targets_met = 0
        for field, target in targets.items():
            try: 
                if int(row[field]) <= target:
                    summary[field] += 1
                    targets_met += 1
            except ValueError as e:
                # This is because the value is 'never done'
                continue
        if targets_met == 3:
            summary['All'] += 1
        summary['total'] += 1

print_summary(summary)

print('Finished!')

Months since Hb A1C : 971 / 1218 ( 79.72 %)
Months since LDL : 1001 / 1218 ( 82.18 %)
Months since Systolic BP [BP:] : 1028 / 1218 ( 84.4 %)
All : 823 / 1218 ( 67.57 %)
total : 1218 / 1218 ( 100.0 %)
Finished!


In [34]:
#! /usr/bin/env python3

# This script analyses the lung health data produced by the search
# MOH - Diabetes - Performance Measures

summary = {
    'coded': 0,
    'total': 0
}

target_codes = ('491', '492', '493', '496')

filename = getfile(title="Select Lung Health Report")

with open(filename) as file:
    next(file)
    
    reader = csv.DictReader(file)
    
    for row in reader:
        codes = [code.strip() for code in row['ICD-9'].split(',')]
        if any(code in codes for code in target_codes):
            summary['coded'] += 1
    
        summary['total'] += 1
    
print_summary(summary)
            
print('Finished')
            

total : 1449 / 1449 ( 100.0 %)
coded : 391 / 1449 ( 26.98 %)
Finished


In [10]:
min_age = 2.5
member_status = 'FHO Enrolled'

def convert_age_to_years(age):
    if age.endswith('mo'):
        months = age.split()[0]
        return int(months) / 12
    else:
        return int(age)
    


30

In [38]:
# Acute and Episodic
from collections import defaultdict
from datetime import datetime

excluded = [
    '1105 - Urgent Care',
    '1105 - Diabetic Check',
    '1105 - Health Review',
    '1105 - WBC',
    'BFARB-WBC',
    'BFAR-Diabetic Check',
    '1105 - Footcare Visit',
    '1105 - KIDS CPX', # CPX = Physical
    '1105 - NP KIDS CPX',
    '1105 - NP WBC',
    '1105 - PN CPX (8011wk!)',
    '1105 - PN(monthly)',
    '1105 - PN(under 8 wks)',
    'BFARB-CPX-F',
    'BFARB-CPD-M',
    'BFARB-KIDS CPX',
]

included = defaultdict(int)

filename = getfile(title="Select appointments report")

assert(filename is not None)

appointments = defaultdict(list)
appointments_YTD = defaultdict(list)

with open(filename, encoding='latin-1') as file:
    
    reader = csv.DictReader(file, delimiter='\t')

    for row in reader:
        if row['Type'] not in excluded:
            if datetime.strptime(row['Date'], '%b %d, %Y') >= datetime(2016,4,1):
                appointments_YTD[row['Patient #']].append(row['Type'])
            if datetime.strptime(row['Date'], '%b %d, %Y') >= datetime(2016,10,1):
                appointments[row['Patient #']].append(row['Type'])
            included[row['Type']] += 1

print('Total Patients: ', len(appointments.keys()))
print('Number of Appointments: ', sum(len(v) for v in appointments.values()))

print('Total Patients (YTD): ', len(appointments_YTD.keys()))
print('Number of Appointments (YTD): ', sum(len(v) for v in appointments_YTD.values()))

print('Included Appointments: ')
for k, v in sorted(included.items()):
    if k == '':
        k = '(blank)'
    print(k + ":", v)
    




Total Patients:  8567
Number of Appointments:  22418
Total Patients (YTD):  8567
Number of Appointments (YTD):  22418
Included Appointments: 
(blank): 18506
1105 - Counselling: 11
1105 - Drivers Medical: 25
1105 - House Call: 18
1105 - Hypertension Followup: 3
1105 - IUD Insertion: 4
1105 - IUD NP Insertion: 5
1105 - MEDREC: 31
1105 - MOCA/MMSE: 36
1105 - PN CPX (8-11wks!): 39
1105 - Pap only: 168
1105 - Pharmacist: 48
1105 - Post Hospital Followup: 42
1105 - Pre-Op: 219
1105 - SW - Followup: 141
1105 - SW - Intial : 119
1105 - Smoke Initial: 27
1105 - SmokeFollowup: 66
1105 - Spirometry F/U with RN: 27
1105 - Spirometry Testing: 53
1105 - Telephone Advice: 2531
1105 - Travel Advice: 23
1105 - Wound Care: 4
1105 -New Patient: 59
BFARB-CPX-M: 54
BFARB-FORMS: 24
BFARB-RECALL: 2
DIET-45 Followup: 38
DIET-Initial Assessment: 85
IFG Initial Counselling: 2
INR: 1
SW Appt: 6
Welsh Physical: 1


In [55]:
# OMSC
from collections import defaultdict
from pprint import pprint

filename = getfile(title="Select appointments report")

assert(filename is not None)

appointments = defaultdict(list)

with open(filename, encoding='latin-1') as file:
    
    reader = csv.DictReader(file, delimiter='\t')

    for row in reader:
        if row['Type'] == '1105 - SmokeFollowup':
            if row['No Show'] == 'T':
                appointments[row['Patient #']].append(False)
            else:
                appointments[row['Patient #']].append(True)

print('Total Patients: ', len(appointments.keys()))
print('Shows: ', len([a for a in appointments.values() if any(a)]))

Total Patients:  81
Shows:  72


In [53]:
len([a for a in appointments.values() if any(a)])

37

In [58]:
print(sorted([int(a) for a in appointments.keys()]))

[155, 544, 832, 1545, 1627, 1989, 2039, 2488, 2624, 2727, 3149, 3481, 4151, 4208, 4308, 4321, 4935, 4952, 4954, 5050, 5243, 5411, 5412, 6279, 6543, 8101, 8182, 8869, 10047, 10362, 11558, 11742, 13953, 14571, 14965, 15619, 15973, 17601, 18003, 18017, 18428, 19437, 21862, 22406, 23127, 24088, 25731, 26148, 26546, 26564, 26945, 27068, 27070, 27370, 27486, 27574, 27628, 27977, 28294, 28602, 28694, 28806, 29698, 29854, 30333, 31229, 31901, 32947, 33230, 33507, 34040, 35564, 36658, 38031, 39124, 39177, 39344, 39472, 39637, 39697, 39921]


In [67]:
temp = set(appointments.keys())

filename = getfile(title="Select appointments report")

ready = []
with open(filename, encoding='latin-1') as file:
    reader = csv.reader(file)
    next(reader)
    next(reader)
    for row in reader:
        ready.append(row[0])

print(ready)
print(temp)

print("Number Patients w Appointment: ", len(temp))
print("Number of Patients Ready to Quit: ", len(ready))

print("Number of Patients in Common: ", len(temp & set(ready)))

['181', '215', '371', '927', '1184', '1216', '1545', '1627', '1718', '1895', '2026', '2488', '3304', '3701', '4308', '4331', '4935', '4952', '5349', '5412', '5626', '5627', '6743', '6996', '7213', '7500', '8182', '8189', '8338', '8471', '8791', '9189', '10956', '11334', '11596', '12311', '14552', '14571', '15767', '15916', '15973', '16795', '18442', '19437', '20262', '20440', '20548', '22340', '23479', '25731', '26004', '26673', '27068', '27370', '27897', '27963', '28155', '28247', '28806', '29698', '30829', '30968', '32769', '33761', '34635', '35225', '35376', '35564', '38554', '39344', '39472', '39486', '39535', '39557', '39637', '39921']
{'14571', '4952', '39344', '5412', '39921', '11742', '26564', '21862', '27977', '38031', '17601', '23127', '3481', '832', '26148', '18428', '29698', '27068', '39637', '4151', '18017', '33230', '28602', '27370', '544', '4308', '4935', '8101', '155', '39124', '15973', '31229', '31901', '30333', '4321', '27070', '6543', '18003', '32947', '8869', '36658

In [62]:
ready = sorted([int(a) for a in appointments.keys()])

set(reader)

['Patient #',
 '181',
 '215',
 '371',
 '927',
 '1184',
 '1216',
 '1545',
 '1627',
 '1718',
 '1895',
 '2026',
 '2488',
 '3304',
 '3701',
 '4308',
 '4331',
 '4935',
 '4952',
 '5349',
 '5412',
 '5626',
 '5627',
 '6743',
 '6996',
 '7213',
 '7500',
 '8182',
 '8189',
 '8338',
 '8471',
 '8791',
 '9189',
 '10956',
 '11334',
 '11596',
 '12311',
 '14552',
 '14571',
 '15767',
 '15916',
 '15973',
 '16795',
 '18442',
 '19437',
 '20262',
 '20440',
 '20548',
 '22340',
 '23479',
 '25731',
 '26004',
 '26673',
 '27068',
 '27370',
 '27897',
 '27963',
 '28155',
 '28247',
 '28806',
 '29698',
 '30829',
 '30968',
 '32769',
 '33761',
 '34635',
 '35225',
 '35376',
 '35564',
 '38554',
 '39344',
 '39472',
 '39486',
 '39535',
 '39557',
 '39637',
 '39921']